In [56]:
import soundfile as sf
import numpy as np
import requests
import librosa
import json

In [2]:
from trankit import Pipeline
# initialize a pipeline for English
p = Pipeline('vietnamese')

Loading pretrained XLM-Roberta, this may take a while...
Loading tokenizer for vietnamese
Loading tagger for vietnamese
Loading lemmatizer for vietnamese
Loading NER tagger for vietnamese
Active language: vietnamese
{'text': 'Hello! This is Trankit.', 'sentences': [{'id': 1, 'text': 'Hello!', 'tokens': [{'id': 1, 'text': 'Hello', 'dspan': (0, 5), 'span': (0, 5)}, {'id': 2, 'text': '!', 'dspan': (5, 6), 'span': (5, 6)}], 'dspan': (0, 6)}, {'id': 2, 'text': 'This is Trankit.', 'tokens': [{'id': 1, 'text': 'This', 'dspan': (7, 11), 'span': (0, 4)}, {'id': 2, 'text': 'is', 'dspan': (12, 14), 'span': (5, 7)}, {'id': 3, 'text': 'Trankit', 'dspan': (15, 22), 'span': (8, 15)}, {'id': 4, 'text': '.', 'dspan': (22, 23), 'span': (15, 16)}], 'dspan': (7, 23)}], 'lang': 'vietnamese'}


In [5]:
# a non-empty string to process, which can be a document or a paragraph with multiple sentences
paragraph = """
Đài truyền thanh thị trấn Rừng Thông xin kính chào toàn thể nhân dân, mời toàn thể nhân dân đón nghe, chương trình phát thanh hôm nay gồm những nội dung sau:
1.	Bài tuyên truyền chung sức xây dựng đô thị văn minh. 
2.	Bản tin tuyên truyền về phát động toàn dân tham gia phong trào: Nhà tôi có bình chữa cháy trên địa bàn thị trấn Rừng Thông.
3.	Thông báo tuyên truyền Hụi, họ, biêu, phường và những hệ lụy khôn lường.
4.	Thông báo đấu giá Quyền sử dụng đất gồm 51 lô đất ở tại khu dân cư thị trấn Rừng Thông, huyện Đông Sơn, Mặt bằng quy hoạch số 4132, 2742, 1879.
5.	Thông báo Thanh niên khám tuyển nghĩa vụ quân sự tại huyện năm 2024.
Sau đây là nội dung chi tiết.
"""
tokenized_doc = p.tokenize(paragraph)

sentences = [sentence["text"] for sentence in tokenized_doc["sentences"]]


In [6]:
sentences

['Đài truyền thanh thị trấn Rừng Thông xin kính chào toàn thể nhân dân, mời toàn thể nhân dân đón nghe, chương trình phát thanh hôm nay gồm những nội dung sau:',
 '1.',
 'Bài tuyên truyền chung sức xây dựng đô thị văn minh.',
 '2.',
 'Bản tin tuyên truyền về phát động toàn dân tham gia phong trào: Nhà tôi có bình chữa cháy trên địa bàn thị trấn Rừng Thông.',
 '3.',
 'Thông báo tuyên truyền',
 'Hụi, họ, biêu, phường và những hệ lụy khôn lường.',
 '4.',
 'Thông báo đấu giá Quyền sử dụng đất gồm 51 lô đất ở tại khu dân cư thị trấn Rừng Thông, huyện Đông Sơn, Mặt bằng quy hoạch số 4132, 2742, 1879.',
 '5.',
 'Thông báo Thanh niên khám tuyển nghĩa vụ quân sự tại huyện năm 2024.',
 'Sau đây là nội dung chi tiết.']

In [57]:
from underthesea import sent_tokenize
from norm.src.processing import run
from trankit import Pipeline
from multiprocessing import Pool

In [58]:
paragraph = """
Đài truyền thanh thị trấn Rừng Thông xin kính chào toàn thể nhân dân, mời toàn thể nhân dân đón nghe, chương trình phát thanh hôm nay gồm những nội dung sau:
1.	Bài tuyên truyền chung sức xây dựng đô thị văn minh. 
2.	Bản tin tuyên truyền về phát động toàn dân tham gia phong trào: Nhà tôi có bình chữa cháy trên địa bàn thị trấn Rừng Thông.
3.	Thông báo tuyên truyền Hụi, họ, biêu, phường và những hệ lụy khôn lường.
4.	Thông báo đấu giá Quyền sử dụng đất gồm 51 lô đất ở tại khu dân cư thị trấn Rừng Thông, huyện Đông Sơn, Mặt bằng quy hoạch số 4132, 2742, 1879.
5.	Thông báo Thanh niên khám tuyển nghĩa vụ quân sự tại huyện năm 2024.
Sau đây là nội dung chi tiết.
"""

In [59]:
def segment_doc(document, max_chunk_size=24, max_num_sent=1):
    chunks = []
    
    tokenized_doc = sent_tokenize(document)
    current_sent = tokenized_doc[0]
    current_length, current_num_sent = len(current_sent.split()), 1
    for sent in tokenized_doc[1:]:
        temp = f'{current_sent} {sent}'
        
        if len(temp.split()) > max_chunk_size or current_num_sent >= max_num_sent:
            chunks.append(current_sent)
            
            current_sent = sent
            current_length = len(sent.split())
            current_num_sent = 1
        else:
            current_sent = temp
            current_length += len(temp.split())
        
        current_num_sent += 1
    
    if chunks[-1] != current_sent:
        chunks.append(current_sent)
        
    return chunks

docs = segment_doc(paragraph)
docs

['\nĐài truyền thanh thị trấn Rừng Thông xin kính chào toàn thể nhân dân, mời toàn thể nhân dân đón nghe, chương trình phát thanh hôm nay gồm những nội dung sau:\n1.',
 'Bài tuyên truyền chung sức xây dựng đô thị văn minh.',
 '2.',
 'Bản tin tuyên truyền về phát động toàn dân tham gia phong trào: Nhà tôi có bình chữa cháy trên địa bàn thị trấn Rừng Thông.',
 '3.',
 'Thông báo tuyên truyền Hụi, họ, biêu, phường và những hệ lụy khôn lường.',
 '4.',
 'Thông báo đấu giá Quyền sử dụng đất gồm 51 lô đất ở tại khu dân cư thị trấn Rừng Thông, huyện Đông Sơn, Mặt bằng quy hoạch số 4132, 2742, 1879.',
 '5.',
 'Thông báo Thanh niên khám tuyển nghĩa vụ quân sự tại huyện năm 2024.',
 'Sau đây là nội dung chi tiết.']

In [60]:
def infer(text, sample_rate=22050):
    text = run(text)
        
    mel = requests.post(
        url='http://127.0.0.1:9090/predictions/encoder',
        json={
            "text": f'{text}'
        }
    )
    
    wav = requests.post(
        url='http://127.0.0.1:9090/predictions/vocoder',
        json={
            "mel": json.dumps(json.loads(mel.content))
        }
    )
    wav = np.array(json.loads(wav.content))[0]
    
    return wav, text

In [61]:
from concurrent.futures import ProcessPoolExecutor, as_completed

docs = segment_doc(paragraph)

with ProcessPoolExecutor(max_workers=1) as executor:
    futures = (executor.submit(infer, doc) for doc in docs)

    wavs = []
    for future in as_completed(futures):
        wav = future.result()
        wavs.append(wav)

đài truyền thanh thị trấn rừng thông xin kính chào toàn thể nhân dân , mời toàn thể nhân dân đón nghe , chương trình phát thanh hôm nay gồm những nội dung sau một .


bài tuyên truyền chung sức xây dựng đô thị văn minh .
hai .
bản tin tuyên truyền về phát động toàn dân tham gia phong trào nhà tôi có bình chữa cháy trên địa bàn thị trấn rừng thông .
ba .
thông báo tuyên truyền hụi , họ , biêu , phường và những hệ lụy khôn lường .
bốn .
thông báo đấu giá quyền sử dụng đất gồm năm mươi mốt lô đất ở tại khu dân cư thị trấn rừng thông , huyện đông sơn , mặt bằng quy hoạch số bốn nghìn một trăm ba mươi hai , hai nghìn bảy trăm bốn mươi hai , một nghìn tám trăm bảy mươi chín .
năm .
thông báo thanh niên khám tuyển nghĩa vụ quân sự tại huyện năm hai nghìn không trăm hai mươi bốn .
sau đây là nội dung chi tiết .


In [62]:
waveform = np.concatenate(wavs)
sf.write("test.wav", waveform, samplerate=22050)